In [2]:
import pandas as pd 

In [1]:
file_path = 'ИТМО_кейс_обучающая_выборка.csv'
data = pd.read_csv(file_path)

In [3]:
print(data.isnull().sum())

Unnamed: 0                    0
Задача                        0
Обстановка                    0
Оптимальный план              0
Задача en                     0
Обстановка en                 0
Оптимальный план en           0
Предсказанный план            0
Успех предсказанного плана    0
dtype: int64


In [4]:
print(data.head())

   Unnamed: 0                         Задача  \
0           0  Приготовить горячий шоколад.    
1           1    Заказать доставку еды домой   
2           2   Реставрация старого комода\n   
3           3           Приготовить сырники.   
4           4              Нарезать колбасу.   

                                          Обстановка  \
0  кухня, молоко, какао, сахар, кастрюля, ложка, ...   
1  мобильный телефон, ресторанное меню, интернет,...   
2  Старый комод, повреждения, шпатель, шкурка, ма...   
3  Кухня, столешница, творог, мука, яйцо, сахар, ...   
4  Кухня, колбаса, нож, доска для резки, холодиль...   

                                    Оптимальный план  \
0  Откройте дверь. Зайдите на кухню. Закрйоте две...   
1  Найдите стол. Подойдите к столу. Найдите мобил...   
2  Подойдите к комоду.Найдите шкурку.Возьмите шку...   
3  Подойти к столешнице. Найти творог. Взять твор...   
4  Подойти к холодильнику. Открыть холодильник. Н...   

                                Задач

In [5]:
print(data['Успех предсказанного плана'].value_counts())

Успех предсказанного плана
1    3234
0    1600
Name: count, dtype: int64


In [6]:
print(data.describe())

        Unnamed: 0  Успех предсказанного плана
count  4834.000000                 4834.000000
mean   2416.500000                    0.669011
std    1395.599931                    0.470618
min       0.000000                    0.000000
25%    1208.250000                    0.000000
50%    2416.500000                    1.000000
75%    3624.750000                    1.000000
max    4833.000000                    1.000000


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Задача'])

In [9]:
print(X.shape)

(4834, 5402)


In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(words)

data['Задача'] = data['Задача'].apply(preprocess_text)
data['Описание'] = data['Описание обстановки'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zamotaevrodion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zamotaevrodion/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zamotaevrodion/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/zamotaevrodion/nltk_data'
    - '/Users/zamotaevrodion/opt/anaconda3/envs/case_for_olimp/nltk_data'
    - '/Users/zamotaevrodion/opt/anaconda3/envs/case_for_olimp/share/nltk_data'
    - '/Users/zamotaevrodion/opt/anaconda3/envs/case_for_olimp/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [11]:
# Инициализация векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000) 

# Объединяем текстовые данные для векторизации
combined_text = data['Задача_processed'] + " " + data['Описание_processed']
X = tfidf_vectorizer.fit_transform(combined_text)

# Целевая переменная
y = data['Успех предсказанного плана']

KeyError: 'Задача_processed'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Объединение текстовых полей
data['combined_text'] = data['Задача en'] + ' ' + data['Обстановка en'] + ' ' + data['Предсказанный план']

# Функция для очистки и нормализации текста
def clean_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление специальных символов
    return text

# Применение функции очистки к каждой строке объединенного текстового поля
data['cleaned_text'] = data['combined_text'].apply(clean_text)

# Подготовка данных и целевой переменной
X = data['cleaned_text']
y = data['Успех предсказанного плана']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация текста с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Ограничение на количество признаков для упрощения модели
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((3867, 1000), (967, 1000))

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Обучение модели логистической регрессии
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_tfidf, y_train)

# Предсказание на тестовой выборке
y_pred = log_reg.predict(X_test_tfidf)

# Расчет точности модели
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.6763185108583247